In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
import pyspark.sql.functions as F

In [0]:
#Leemos un dataframe

# Leer un CSV 
simpleMEMBERS = (spark
  .read
  .option("header", True) #forma de lectura// trae la cabecera
  .csv("/FileStore/tables/members.csv") ###conexion de azure
  #.sample(withReplacement=False, fraction=0.3, seed=3) # cargamos solo una muestra de la data
           )
# Mostrando el contenido de la data
display(simpleMEMBERS)
#simpleMEMBERS.show()

customer_id,join_date
A,2021-01-07
B,2021-01-09


In [0]:
simpleMENU = (spark
    .read
    .option("header", True)
    .csv("/FileStore/tables/menu.csv")
              )

display(simpleMENU)

product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


In [0]:
simpleSALES = (spark
               .read
               .option("header", True)
               .csv("/FileStore/tables/sales.csv")


)

display(simpleSALES)

customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [0]:
simpleMENU.createOrReplaceTempView("menu")
simpleMEMBERS.createOrReplaceTempView("members")
simpleSALES.createOrReplaceTempView("sales")

In [0]:
sqlunion = spark.sql('''select * from members''')

In [0]:
display(sqlunion) #memebers

customer_id,join_date
A,2021-01-07
B,2021-01-09


In [0]:
#1. ¿Cuál es la cantidad total que gastó cada cliente en el restaurante?

In [0]:
pregunta1 = (
    simpleSALES.join(simpleMENU, simpleSALES.product_id ==  simpleMENU.product_id, "inner")
    .groupBy('customer_id')
    .agg(
        F.sum('price').alias('Total gasto')
    )
    .orderBy(F.col('Total gasto').desc())
)

display(pregunta1)

customer_id,Total gasto
A,76.0
B,74.0
C,36.0


In [0]:
#2. ¿Cuántos días ha visitado cada cliente el restaurante?

In [0]:
display(simpleSALES)

customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2
A,2021-01-10,3
A,2021-01-11,3
A,2021-01-11,3
B,2021-01-01,2
B,2021-01-02,2
B,2021-01-04,1
B,2021-01-11,1


In [0]:
pregunta2 = simpleSALES.groupby('customer_id').agg(F.countDistinct('order_date').alias('Dias Visita'))
display(pregunta2)

customer_id,Dias Visita
B,6
C,2
A,4


In [0]:
#3. ¿Cuál fue el primer artículo del menú comprado por cada cliente?

In [0]:
from pyspark.sql.window import Window

pregunta3 = (
    simpleSALES
    .join(simpleMENU,simpleMENU.product_id == simpleSALES.product_id,'inner')
    .withColumn('row_number', F.row_number().over(
        Window
        .partitionBy('customer_id')
        .orderBy('order_date')
    )
               )
    .filter('row_number == 1')
    .select('customer_id', 'order_date', 'product_name')
    .withColumnRenamed('product_name', 'primer_product')
    .withColumnRenamed('order_date', 'fecha_primer_pedido')
)

display(pregunta3)

customer_id,fecha_primer_pedido,primer_product
A,2021-01-01,sushi
B,2021-01-01,curry
C,2021-01-01,ramen


In [0]:
#4. ¿Cuál es el artículo más comprado en el menú y cuántas veces lo compraron todos los clientes?

In [0]:
pregunta4 = (
    simpleSALES
    .join(simpleMENU,simpleMENU.product_id == simpleSALES.product_id,'inner'
    )
    .groupBy('product_name')
    .agg(
        F.count('order_date').alias('product_count')
    )
    .orderBy(F.col('product_count').desc())
)

pregunta4.show(1)

+------------+-------------+
|product_name|product_count|
+------------+-------------+
|       ramen|            8|
+------------+-------------+
only showing top 1 row



In [0]:
#5. ¿Qué artículo fue el más popular para cada cliente?

In [0]:
pregunta5 = (
    simpleSALES
    .join(
        simpleMENU,
        simpleMENU.product_id == simpleSALES.product_id,
        'inner'
    )
    .groupBy('customer_id', 'product_name')
    .agg(
        F.count('product_name').alias('orders')
    )
    .withColumn('plate_rank', F.rank().over(
        Window
        .partitionBy('customer_id')
        .orderBy(F.col('orders').desc())
    )
               )
    .filter('plate_rank == 1')
)

display(pregunta5)

customer_id,product_name,orders,plate_rank
A,ramen,3,1
B,sushi,2,1
B,ramen,2,1
B,curry,2,1
C,ramen,3,1


In [0]:
#6. ¿Qué artículo compró primero el cliente después de convertirse en miembro?

In [0]:
simpleMENU.createOrReplaceTempView("menu")
simpleMEMBERS.createOrReplaceTempView("members")
simpleSALES.createOrReplaceTempView("sales")

In [0]:
sql1 = """
WITH joined_tables AS (
    SELECT 
          s.*
        , m.join_date
        , menu.product_name
    FROM sales AS s
    INNER JOIN members AS m
        ON s.customer_id = m.customer_id
        AND s.order_date >= m.join_date
    INNER JOIN menu
    ON menu.product_id = s.product_id
)
SELECT *
, ROW_NUMBER() OVER(PARTITION BY customer_id order by order_date) AS rn
FROM joined_tables
"""

pregunta6 = (
    spark
    .sql(sql1)
    .filter('rn == 1')
    .select(['customer_id', 'order_date', 'join_date', 'product_name'])
)

display(pregunta6)

customer_id,order_date,join_date,product_name
A,2021-01-07,2021-01-07,curry
B,2021-01-11,2021-01-09,sushi


In [0]:
#7.¿Qué artículo se compró justo antes de que el cliente se convirtiera en miembro?

In [0]:

sql2 = """
                select customer_id, 
                       order_date, 
                       product_id, 
                       join_date from (
                select s.*, join_date, ROW_NUMBER() OVER(PARTITION BY s.customer_id
                ORDER BY order_date DESC) AS Row_Number from sales s inner join members m on s.customer_id = m.customer_id
                and order_date < join_date
                ) where Row_Number = 1
"""

pregunta7 = (
    spark
    .sql(sql2)
    .select(['customer_id', 'order_date',  'product_id',  'join_date'])
)

display(pregunta7)

customer_id,order_date,product_id,join_date
A,2021-01-01,1,2021-01-07
B,2021-01-04,1,2021-01-09


In [0]:
#8.¿Cuál es el total de artículos y la cantidad gastada por cada miembro antes de convertirse en miembro?

In [0]:

all_details_query = """
SELECT
      s.customer_id
    , s.order_date
    , s.product_id
    , menu.product_name
    , menu.price
    , m.join_date
FROM sales AS s
    LEFT JOIN members AS m
        ON m.customer_id = s.customer_id
    LEFT JOIN menu
        ON menu.product_id = s.product_id
"""

all_details_df = (
    spark
    .sql(all_details_query)
)

all_details_df.createOrReplaceTempView('all_details')


pregunta8 = (
    all_details_df
    .filter(F.col('join_date').isNotNull())
    .filter('order_date < join_date')
    .groupBy('customer_id')
    .agg(
        F.count('product_id').alias('count_items'),
        F.sum('price').alias('money_spend')
    )
)

display(pregunta8)

customer_id,count_items,money_spend
B,3,40.0
A,2,25.0


In [0]:
#9.¿Si cada $ 1 gastado equivale a 10 puntos y el sushi tiene un multiplicador de puntos 2x, ¿cuántos puntos tendría cada cliente?

In [0]:
sql9 = """
WITH points_cte AS (
    SELECT 
          *
        , CASE 
            WHEN product_name = 'sushi' THEN (price * 10 * 2)
            ELSE price * 10 END AS points 
    FROM all_details
)
SELECT
      customer_id
    , SUM(points) AS total_points
FROM points_cte
GROUP BY customer_id
ORDER BY total_points DESC;
"""

pregunta9 = (
    spark
    .sql(sql9)
)

display(pregunta9)

customer_id,total_points
B,940.0
A,860.0
C,360.0


In [0]:
#10.¿En la primera semana después de que un cliente se une al programa (incluida la fecha de ingreso), gana el doble de puntos en todos los artículos, no solo en sushi. ¿Cuántos puntos tienen los clientes A y B a fines de enero?

In [0]:
sql10 = """
WITH points_cte AS (
    SELECT 
          *
        , CASE
           
            WHEN order_date < join_date AND product_name = 'sushi' THEN (price * 10 * 2)
            WHEN order_date < join_date AND product_name != 'sushi' THEN (price * 10)
          
            WHEN order_date BETWEEN join_date and join_date + INTERVAL '7 days' THEN (price * 10 * 2)
           
            WHEN product_name = 'sushi' THEN (price * 10 * 2)
            ELSE price * 10 END AS points 
    FROM all_details
    WHERE
        customer_id IN ('A', 'B')

        AND order_date <= '2021-01-31'
)
SELECT
      customer_id
    , SUM(points) AS total_points
FROM points_cte
GROUP BY customer_id
ORDER BY total_points DESC;
"""

pregunta10 = (
    spark
    .sql(sql10)
)

display(pregunta10)

customer_id,total_points
A,1370.0
B,940.0
